### 아이디어
- 구간 합 트리를 이용해서 풀면 될 것 같다.
- 1번 쿼리를 하는데에 있어, [a, b] 구간 사이에 모든 DVD가 존재한다면, a에서 b까지 1씩 증가하면서 누적된 값이 나올 것이다.
  - ex) Q2(3, 5) => 3 + 4 + 5 = 12 (DVD가 존재하는 상태)
- 매번 sum(range(a, b+1))을 할 순 없을테니, 세그트리를 한개 더 만들어놓아서 그것과 비교하면 될 것 같다.
- 0-indexed 문제임에 유의

### 반례
- 2 1 3 5 4 인 상태라고 했을 때 Q2(1, 3)는 YES라고 답할 것이다

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class SegmentTree:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, idx):
    self[idx] = self._def

  def __getitem__(self, idx):
    return self.L[idx + self._size]

  def __setitem__(self, idx, value):
    idx += self._size
    self.L[idx] = value
    idx >>= 1
    while idx:
      self.L[idx] = self._f(self.L[2 * idx], self.L[2 * idx + 1])
      idx >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol() :
  N, K = map(int, input().split())
  sg1 = SegmentTree([*range(N)], 100001, min)
  sg2 = SegmentTree([*range(N)], -1, max)
  for _ in range(K) :
    Q, A, B = map(int, input().split())
    if Q == 0 :
      sg1[A], sg1[B] = sg1[B], sg1[A]
      sg2[A], sg2[B] = sg2[B], sg2[A]
    elif Q == 1 :
      answer = sg1.query(A, B+1) == A and sg2.query(A, B+1) == B
      sys.stdout.write("YES\n" if answer else "NO\n")

for _ in range(int(input())) :
  sol()

### 풀이
- 최대/최소를 구하는 세그트리를 만들고, 그것을 쿼리로 비교한다. 구간 [A, B]에 모든 책이 존재한다고 할 경우
  - A는 출력 쿼리에서 작은값이 min 세그의 값과 같을 것이며
  - B는 출력 쿼리에서 큰값이 max 세그의 값과 같을 것이다.
- 구간곱과 구간합을 비교 분석해서 푼 사람도 있다고한다.

### 증명(귀류법)
- 최소/최대 세그트리는 구간내의 최소/최대 값을 올바르게 반환한다고 가정하며, 정당성 증명은 생략한다.
- 만약 입력으로 주워진 쿼리 `A, B`가 정답이 아니라고 한다면 2가지 경우를 고려할 수 있다.
    1. 쿼리구간 `[A, B]` 사이에 최소/최대 값이 `A, B`로 올바르게 나왔는데, 모든 DVD가 존재하지 않는 경우
        - 이는 구간 `[A, B]` 사이에 적어도 1개 이상의 DVD가 없다는 의미이다.
        - `[A, B]`는 `B - A + 1`개를 가지고 있고, 비둘기집의 원리에 의해 다른 쿼리 구간에 중복되는 DVD번호가 존재한다는 의미가 된다.
        - 모든 DVD 번호는 유일하다고 문제에 정의돼있으므로 모순이 된다.
    2. 모든 DVD가 구간 `[A, B]`에 존재하는데, 최소/최대 값이 `A, B`가 아닌 경우
        - 이는 선반에 A보다 작은 DVD번호가 있거나, B보다 큰 DVD번호가 있다는 의미이다.
        - 하지만 A부터 B까지 모든 DVD는 각각 한개씩 존재하므로, `[A, B]` 이외에 DVD 번호가 존재할 순 없다.
        - 최소/최대 세그트리가 올바른 값을 반환한다고 가정했으므로, 이 가정에 모순된다.
    